# `diurnal` Library Demonstration

This notebook shows how to use the `diurnal` library to determine RNA secondary structures with neural networks.

## Obtain Data

The module `diurnal.database` can download the following RNA databases:

| Database name  | Number of molecules | URL of the original database
| -------------- | ------------------- | ------------------------------
| archiveII      | 3975 | https://rna.urmc.rochester.edu/pub/archiveII.tar.gz |
| RNASTRalign    | 37149 | https://www.urmc.rochester.edu/rna/ |
| RNA_STRAND     | 4666 | http://www.rnasoft.ca/strand/ |

The code:

In [ ]:
from diurnal import database

database.download("./data/", "archiveII")

downloads the `archiveII` dataset in the directory `./data/`. RNA molecules are described with two structures:

- The **primary structure** is a *sequence* of nuclotides, which is represented by a string of four characters: `A`, `C`, `G`, and `U`.
- The **secondary structure** describes the *pairing* of nucleotides, which is represented in the databases by a list of indices. The value `0` indicates that a nucleotide is unpaired.

## Format Data

Since neural networks use numerical vectors as inputs and outpus, the content of RNA databases must be converted into numerical types for training.

The **primary structure** (for instance, `AAACCCUUU`) can be converted to a numerical vector through *one0hot encoding*, which consists in assigning an orthogonal to each different letter.

The **secondary structure** (for instance, `[8, 7, 6, -1, -1, -1, 2, 1, 0]`) can be used as it is, but this representation is difficult to predict because of its high dimentionality. The secondary structure can be converted into simpler representations:

- The **bracket notation** represents unpaired nucleotides with the character `.` and paired nucleotides with the character `(` or `)` depending on whether the paired base is located nearer the 5' or 3' end of the molecule. The bracket notation can then be one-hot encoded.
- the **shadow** represents paired nucleotide with the number `0` and paired nucleotides with the character `1`. This representation does not fully describe the secondary structure and it is simpler to predict.

For example:

In [2]:
from diurnal.structure import Primary, Secondary

primary_structure = "AAACCCUUU"
secondary_structure = [8, 7, 6, -1, -1, -1, 2, 1, 0]

print("One-hot encoded primary structure:")
print(Primary.to_vector(primary_structure))
print()

print("Bracket notation of the secondary structure:")
print(Secondary.to_bracket(secondary_structure))
print()

print("One-hot encoded secondary structure:")
print(Secondary.to_vector(secondary_structure))
print()

print("Shadow of the secondary structure:")
print(Secondary.to_shadow(secondary_structure))
print()

One-hot encoded primary structure:
[[1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]]

Bracket notation of the secondary structure:
['(', '(', '(', '.', '.', '.', ')', ')', ')']

One-hot encoded secondary structure:
[[1 0 0]
 [1 0 0]
 [1 0 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 0 1]
 [0 0 1]
 [0 0 1]]

Shadow of the secondary structure:
[1, 1, 1, 0, 0, 0, 1, 1, 1]



The library can read all the molecules of a database, encode them according to a specific schema, and write the resulting data in reusable files, as shown below:

In [5]:
from diurnal import database, structure

database.format(
    "../data/archiveII",  # Directory of the raw data to format.
    "../data/formatted",  # Formatted data output directory.
    512,  # Normalized size. Short molecules are padded at the 3' end.
    structure.Primary.to_vector,  # Primary to vector map.
    structure.Secondary.to_vector  # Secondary to vector map.
)

2023-07-12T21:59:04.556071 > INFO Format RNA data into Numpy files.
2023-07-12T21:59:05.428057 >     The directory ../data/formatted/ already contains the formatted data.


## Split Data for Training and Testing

Formatted data must be split into training and testing sets to elaborate and test predictive models. One way to do this is to use one RNA family (a group of similar RNA molecules) for testing. This ensures that the model is generic enough to predict the structure of molecules that are different from the ones comprised in the training set. 

In [1]:
from diurnal import train, family

test_set = train.load_families("../data/formatted", "5s")
train_set = train.load_families("../data/formatted", family.all_but("5s"))

2023-07-12T22:02:48.046642 > INFO Loading the family 5s from `../data/formatted`.
2023-07-12T22:02:48.081373 > INFO Loading the families ['16s', '23s', 'grp1', 'grp2', 'RNaseP', 'SRP', 'telomerase', 'tmRNA', 'tRNA'] from `../data/formatted`.


## Train a Model

The library contains examples of predictive models that can be loaded, trained, and evaluated as shown below:

In [ ]:
import torch
from diurnal import models

model = models.NN(
    model=models.networks.cnn.Pairings_1,
    N=512,
    n_epochs=3,
    optimizer=torch.optim.Adam,
    loss_fn=torch.nn.MSELoss,
    optimizer_args={"eps": 1e-4},
    loss_fn_args=None,
    verbosity=1)
model.train(train_set)

In this example above, the object `diurnal.models.NN` is a model that simply wraps a `pytorch` neural network. The model receives as arguments an optimizer and loss function that are used to train the network.

## Predict Structures

Trained models can predict secondary structures from primary structures as follows:

In [ ]:
from diurnal import structure

primary_structure = list("AAAACCCCUUUU")
encoded_primary_structure = structure.Primary.to_vector(primary_structure)
prediction = model.predict(encoded_primary_structure)
print(f"Prediction: {prediction}")

## Evaluate Results

Predictions can be evaluated with two metrics:

- **Recall** (or sensitivity or true positive rate) is the probability that a positive prediction is actually positive.
- **Precision** (or positive predictive value) is the fraction of relevant elements among retrieved elements.

The F1-score is the geometric mean of recall and precision.

In [ ]:
from diurnal import evaluate

true = list("(((....)))")
prediction = list("((......))")
recall, precision, f1 = evaluate.recall_precision_f1(true, prediction)

The library also contains **baseline models** that make very simple predictions to compare results with actual predictive models.

In [ ]:
from diurnal.models import baseline

baseline_model = baseline.Random()
baseline_model.train(train_set)
prediction = baseline_model.predict(encoded_primary_structure)
print(f"Prediction: {prediction}")

## Save Models

Predictive models can be written in the file system to be loaded and used subsequently.

In [ ]:
model.save(directory = "saved_model")

## Load Models

Models can be loaded from the file system to replicate results.

In [ ]:
from diurnal import models

model = models.NN(
   cnn.Pairings_1,
   SIZE,
   None,
   torch.optim.Adam,
   torch.nn.MSELoss,
   {"eps": 1e-4},
   None,
   verbosity=1)
model.load("saved_model")

## Visualize Results

The library contains a data visualization module to view results.

In [ ]:
from diurnal import visualize

print(f"\nSample prediction from the test set (`{test_set['names'][0]}`).")
p = test_set["primary_structures"][0]
s = test_set["secondary_structures"][0]
visualize.prediction(p, s, loaded_model.predict(p))